<a href="https://colab.research.google.com/github/AbdullahAlTalaq/CodeForces-/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install inference_sdk

  Using cached supervision-0.25.1-py3-none-any.whl.metadata (14 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 11.7 MB/s eta 0:00:00
Using cached supervision-0.25.1-py3-none-any.whl (181 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.11.0.86
    Uninstalling opencv-python-4.11.0.86:
      Successfully uninstalled opencv-python-4.11.0.86
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.11.15
    Uninstalling aiohttp-3.11.15:
      Successfully uninstalled aiohttp-3.11.15


In [4]:
!pip install supervision

In [5]:
!git clone https://github.com/roboflow/sports.git
!pip install umap-learn scikit-learn torch torchvision

import sys
sys.path.append('/content/sports')

from sports.common.team import TeamClassifier





fatal: destination path 'sports' already exists and is not an empty directory.


In [7]:
from inference_sdk import InferenceHTTPClient
import cv2
import supervision as sv
from sports.common.team import TeamClassifier
import os
import time
import numpy as np
from tqdm import tqdm

# --- Constants ---
BALL_ID = 0
GOALKEEPER_ID = 1
PLAYER_ID = 2
REFEREE_ID = 3
STRIDE = 30

# --- Roboflow API Client ---
CLIENT = InferenceHTTPClient(
    api_url="https://serverless.roboflow.com",
    api_key="D3Xlf1qhYUAr4LQ84DbN"
)

# --- Video Setup ---
video_path = "/content/sample.mp4"
output_path = "/content/output1.mp4"
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (w, h))

# --- Annotators & Tracker ---
ellipse_annotator = sv.EllipseAnnotator(
    color=sv.ColorPalette.from_hex(['#00BFFF', '#FF1493', '#FFD700']),
    thickness=2
)
triangle_annotator = sv.TriangleAnnotator(
    color=sv.Color.from_hex('#FFD700'),
    base=25,
    height=21,
    outline_thickness=1
)
label_annotator = sv.LabelAnnotator(
    color=sv.ColorPalette.from_hex(['#00BFFF', '#FF1493', '#FFD700']),
    text_color=sv.Color.from_hex('#000000'),
    text_position=sv.Position.BOTTOM_CENTER
)
tracker = sv.ByteTrack()
tracker.reset()

# --- Step 1: Collect Player Crops ---
frame_generator = sv.get_video_frames_generator(video_path, stride=STRIDE)
crops = []

print("Collecting player crops for team classification...")
for frame in tqdm(frame_generator, desc="Collecting crops"):
    result = CLIENT.infer(frame, model_id="football-players-detection-3zvbc/11")
    detections = sv.Detections.from_inference(result)
    players = detections[detections.class_id == PLAYER_ID]
    players_crops = [sv.crop_image(frame, xyxy) for xyxy in players.xyxy]
    crops += players_crops

# --- Step 2: Train Team Classifier ---
team_classifier = TeamClassifier(device="cpu")
team_classifier.fit(crops)

# --- Step 3: Process Video Frame-by-Frame ---
frame_count = 0
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

def resolve_goalkeepers_team_id(players, goalkeepers):
    goalkeepers_xy = goalkeepers.get_anchors_coordinates(sv.Position.BOTTOM_CENTER)
    players_xy = players.get_anchors_coordinates(sv.Position.BOTTOM_CENTER)
    team_0_centroid = players_xy[players.class_id == 0].mean(axis=0)
    team_1_centroid = players_xy[players.class_id == 1].mean(axis=0)
    result = []
    for gk_xy in goalkeepers_xy:
        dist_0 = np.linalg.norm(gk_xy - team_0_centroid)
        dist_1 = np.linalg.norm(gk_xy - team_1_centroid)
        result.append(0 if dist_0 < dist_1 else 1)
    return np.array(result)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    temp_path = "/content/temp_frame.png"
    cv2.imwrite(temp_path, frame)

    result = CLIENT.infer(temp_path, model_id="football-players-detection-3zvbc/11")
    detections = sv.Detections.from_inference(result)

    # Ball
    ball_detections = detections[detections.class_id == BALL_ID]
    ball_detections.xyxy = sv.pad_boxes(xyxy=ball_detections.xyxy, px=10)

    # All other
    others = detections[detections.class_id != BALL_ID]
    others = others.with_nms(threshold=0.5, class_agnostic=True)
    others = tracker.update_with_detections(detections=others)

    # Split by role
    goalkeepers = others[others.class_id == GOALKEEPER_ID]
    players = others[others.class_id == PLAYER_ID]
    referees = others[others.class_id == REFEREE_ID]

    # Predict teams for players
    player_crops = [sv.crop_image(frame, xyxy) for xyxy in players.xyxy]
    players.class_id = team_classifier.predict(player_crops)

    # Assign team to goalkeepers
    goalkeepers.class_id = resolve_goalkeepers_team_id(players, goalkeepers)

    # Mark referees with neutral class (e.g., -1 -> 2 for yellow)
    referees.class_id = np.full_like(referees.class_id, 2)

    # Merge all
    all_detections = sv.Detections.merge([players, goalkeepers, referees])
    labels = [f"#{id}" for id in all_detections.tracker_id]
    all_detections.class_id = all_detections.class_id.astype(int)

    # Annotate
    annotated = frame.copy()
    annotated = ellipse_annotator.annotate(scene=annotated, detections=all_detections)
    annotated = label_annotator.annotate(scene=annotated, detections=all_detections, labels=labels)
    annotated = triangle_annotator.annotate(scene=annotated, detections=ball_detections)

    out.write(annotated)
    frame_count += 1
    print(f"Processed frame {frame_count}")
    time.sleep(1)

cap.release()
out.release()
os.remove(temp_path)
print(f"Saved annotated video to: {output_path}")

The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.


config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/813M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/368 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


...

Embedding extraction: 1it [00:04,  4.91s/it]


Processed frame 529


Embedding extraction: 1it [00:06,  6.75s/it]


Processed frame 530


Processed frame 531
Processed frame 532
Processed frame 533
Processed frame 534
Processed frame 535
Processed frame 536
Processed frame 537
Processed frame 538
Processed frame 539
Processed frame 540
Processed frame 541
Processed frame 542
Processed frame 543
Processed frame 544
Processed frame 545
Processed frame 546
Processed frame 547
Processed frame 548
Processed frame 549
Processed frame 550
Processed frame 551
Processed frame 552
Processed frame 553
Processed frame 554
Processed frame 555
Processed frame 556
Processed frame 557
Processed frame 558
Processed frame 559
Processed frame 560
Processed frame 561
Processed frame 562
Processed frame 563
Processed frame 564
Processed frame 565
Processed frame 566
Processed frame 567
Processed frame 568
Processed frame 569
Processed frame 570
Processed frame 571
Processed frame 572
Processed frame 573
Processed frame 574
Processed frame 575
Processed frame 576
Processed frame 577
Saved annotated video to: /content/output1.mp4
